In [ ]:
from txgraffiti.playground    import ConjecturePlayground
from txgraffiti.generators    import convex_hull, linear_programming, ratios
from txgraffiti.heuristics    import morgan_accept, dalmatian_accept
from txgraffiti.processing    import remove_duplicates, sort_by_touch_count
from txgraffiti.example_data  import integer_data   # bundled toy dataset
from txgraffiti import TRUE
from txgraffiti.example_data  import graph_data   # bundled toy graph dataset
from txgraffiti import Predicate, Constant

ai = ConjecturePlayground(
    graph_data,
    object_symbol='G',
    base='connected',
)

regular = Predicate('regular', lambda df: df['min_degree'] == df['max_degree'])
cubic = regular & (ai.max_degree == 3)


# 4) Run discovery
ai.discover(
    methods         = [ratios, convex_hull, linear_programming],
    features        = [ai.independence_number],
    target          = ai.zero_forcing_number,
    heuristics      = [dalmatian_accept, morgan_accept],
    hypothesis = [(ai.max_degree <= 3) & (~ai.Kn), cubic],
    post_processors = [remove_duplicates, sort_by_touch_count],
)

for idx, conj in enumerate(ai.conjectures[:10], start=1):
    formula = ai.forall(conj)
    print(f"Conjecture {idx}. {formula}\n")

ImportError: cannot import name 'morgan' from 'txgraffiti.heuristics' (/Users/randydavila/Documents/automated-conjecturing/txgraffiti2/txgraffiti/heuristics/__init__.py)

In [ ]:
from txgraffiti.playground    import ConjecturePlayground
from txgraffiti.generators    import convex_hull, linear_programming, ratios
from txgraffiti.heuristics    import morgan, dalmatian
from txgraffiti.processing    import remove_duplicates, sort_by_touch_count
from txgraffiti.example_data  import integer_data   # bundled toy dataset
from txgraffiti import TRUE

# 2) Instantiate your playground
#    object_symbol will be used when you pretty-print "∀ G.connected: …"
ai = ConjecturePlayground(
    integer_data,
    object_symbol='n.PositiveInteger'
)

ai.discover(
    methods         = [convex_hull, linear_programming, ratios],
    features        = ['sum_divisors', 'prime_factor_count'],
    target          = 'collatz_steps',
    heuristics      = [morgan, dalmatian],
    post_processors = [remove_duplicates, sort_by_touch_count],
)

# 5) Print your top conjectures
for idx, conj in enumerate(ai.conjectures[:10], start=1):
    # wrap in ∀-notation for readability
    formula = ai.forall(conj)
    print(f"Conjecture {idx}. {formula}\n")

In [ ]:
import pandas as pd
from txgraffiti.logic.conjecture_logic import Predicate, Conjecture, TRUE, FALSE

# Step 1: Simulate a truth table over 3 variables: A, B, C
from itertools import product

rows = list(product([False, True], repeat=3))
df = pd.DataFrame(rows, columns=["A", "B", "C"])

# Step 2: Create symbolic predicates
A = Predicate("A", lambda df: df["A"])
B = Predicate("B", lambda df: df["B"])
C = Predicate("C", lambda df: df["C"])

# Step 3: Create and test some formulas
formulas = [
    A >> A,
    A >> B,
    A >> (A | B),
    (A & B) >> C,
    (A & ~A),
    (A | ~A),
    (A & B) >> (A | B),
    ((A >> B) & (B >> C)) >> (A >> C),  # hypothetical syllogism
]

# Step 4: Evaluate and print which are tautologies
for f in formulas:
    if f(df).all():
        print(f"{f}  ::  Tautology")

In [ ]:
import pandas as pd
from txgraffiti.logic import Property, TRUE
from txgraffiti.generators.geometric import convex_hull

df = pd.DataFrame({
    "a": [1, 2, 3],
    "b": [2, 4, 8],
    "t": [3, 6, 11]
})

a = Property("a", lambda df: df["a"])
b = Property("b", lambda df: df["b"])
t = Property("t", lambda df: df["t"])

for conj in convex_hull(df, features=[a, b], target=t, hypothesis=TRUE):
    print(conj)

In [ ]:
!pip install networkx

In [ ]:
!pip install graphcalc

In [ ]:
import networkx as nx
import pandas as pd
from txgraffiti.logic import Property, TRUE
from txgraffiti.generators.geometric import convex_hull
from itertools import combinations
from graphcalc import diameter, size  # replace with your actual graph invariant functions

# Generate all connected 4-vertex graphs
paw = nx.Graph()
paw.add_edges_from([(0, 1), (1, 2), (1, 3), (3, 0), (0, 4)])

graphs = [
        nx.star_graph(3),                      # (2, 3)
        nx.path_graph(4),                      # (3, 3)
        paw,         # (2, 3) + isolated node
        nx.cycle_graph(4),                     # (2, 4)
        nx.Graph([(0,1), (1,2), (2,3), (0,2), (1,3)]),  # (2, 5)
        nx.complete_graph(4)                   # (1, 6)
    ]


# Build dataframe of invariants
df = pd.DataFrame({
    "name": [f"G{i}" for i in range(len(graphs))],
    "D": [diameter(G) for G in graphs],
    "m": [size(G) for G in graphs],
    'connected_graph': [True for G in graphs],
})

D = Property("D", lambda df: df["D"])
m = Property("m", lambda df: df["m"])
connected = Predicate("connected graph", lambda df: df["connected_graph"])

for conj in convex_hull(df, features=[D], target=m, hypothesis=connected):
    print(conj)

In [ ]:
G = nx.Graph()
G.add_edges_from([(0, 1), (1, 2), (1, 3), (3, 0), (0, 4)])



In [ ]:
import pandas as pd
from txgraffiti.example_data      import graph_data
from txgraffiti.playground        import ConjecturePlayground
from txgraffiti.logic.conjecture_logic import (
    Property, Predicate, Inequality, Conjecture, Constant
)
from txgraffiti.generators        import ratios, convex_hull, linear_programming


def sophie_accept(
    new_conj: Conjecture,
    existing: list[Conjecture],
    df:       pd.DataFrame,
    *,
    min_fraction: float = 0.10
) -> bool:
    # 1. Perfect precision
    if not new_conj.is_true(df):
        return False

    # 2. Support on dataset
    Hmask = new_conj.hypothesis(df)
    if Hmask.mean() < min_fraction:
        return False

    # 3. Must cover at least one true target
    Tmask = new_conj.conclusion(df)  # this is ~H or T; use T separately
    true_under_H = (new_conj.hypothesis(df) & new_conj.conclusion(df))
    if not true_under_H.any():
        return False

    # 4. Compare only against best existing for same target
    same_target = [
        old for old in existing
        if old.conclusion == new_conj.conclusion
    ]
    if same_target:
        # pick the one with max (recall, -complexity)
        def score(c):
            recall = c.accuracy(df)
            complexity = len(getattr(c.hypothesis, "_and_terms", [c.hypothesis]))
            return (recall, -complexity)
        best_old = max(same_target, key=score)

        # reject if it’s at least as good
        if score(best_old) >= (new_conj.accuracy(df),
                               -len(getattr(new_conj.hypothesis, "_and_terms",
                                            [new_conj.hypothesis]))):
            return False

    return True


# 1) Prepare the DataFrame & session
df = graph_data.copy()
pg = ConjecturePlayground(df, object_symbol="G", base="connected")

# 2) Define the Boolean target predicate
T_bool = Predicate(
    "bipartite",
    lambda df: df["bipartite"]
)

# 3) Cast it to an integer-valued Property (0 or 1)
T_int = Property(
    "bipartite_int",
    lambda df: df["bipartite"].astype(int)
)

# 4) Our numeric features
features = [pg.independence_number, pg.order]

# 5) Helper to translate a numeric conjecture back into Boolean form
def translate_numeric_conj(conj: Conjecture) -> Conjecture:
    # conj.hypothesis : Predicate H
    ineq: Inequality = conj.conclusion  # T_int op rhs
    op = ineq.op
    rhs = ineq.rhs  # a Property
    H  = conj.hypothesis

    # Case 1: T_int >= rhs  → we want H ⇒ T_bool
    if op in (">=", ">"):
        # demand rhs > 0  ⇒ T_int must be 1
        return Conjecture(H, T_bool)

    # Case 2: T_int <= rhs  → H ⇒ ¬T_bool
    if op in ("<=", "<"):
        # demand rhs < 1  ⇒ T_int must be 0
        return Conjecture(H, ~T_bool)

    # otherwise drop it
    return None

# 6) Generate *numeric* conjectures for both positive and negative cases
candidates = []
for target_prop in (T_int, Property("one_minus_hp", lambda df: 1 - df["bipartite"].astype(int))):
    # if target_prop is one_minus_hp, then a >= bound
    # will correspond to ¬T_bool, and similarly <= ⇒ T_bool
    for gen in (ratios, convex_hull, linear_programming):
        for conj in gen(
            df,
            features   = features,
            target     = target_prop,
            hypothesis = pg.base
        ):
            # only keep those that hold exactly on the DataFrame
            if not conj.is_true(df):
                continue
            # translate into Boolean form
            bconj = translate_numeric_conj(conj)
            if bconj is not None:
                candidates.append(bconj)

# 7) Run Sophie to filter
accepted = []
for c in candidates:
    if sophie_accept(c, accepted, df, min_fraction=0.1):
        accepted.append(c)

# 8) Display the final Boolean conjectures
for i, conj in enumerate(accepted, start=1):
    print(f"Conjecture {i}: {pg.forall(conj)}   acc={conj.accuracy(df):.2%}")


In [ ]:
import pandas as pd
from txgraffiti.example_data      import graph_data
from txgraffiti.playground        import ConjecturePlayground
from txgraffiti.logic.conjecture_logic import (
    Property, Predicate, Inequality, Conjecture, Constant
)
from txgraffiti.generators        import ratios, convex_hull, linear_programming
f

# 1) Prepare the DataFrame & session
df = graph_data.copy()

# 2) Materialize the integer proxies
df["bipartite_int"]        = df["bipartite"].astype(int)
df["one_minus_bipartite"]  = 1 - df["bipartite_int"]


pg = ConjecturePlayground(df, object_symbol="G", base="connected")

# 3) Define target Predicates / Properties
T_bool     = Predicate("bipartite", lambda df: df["bipartite"])
T_int      = Property("bipartite_int",      lambda df: df["bipartite_int"])
T_inv_int  = Property("one_minus_bipartite", lambda df: df["one_minus_bipartite"])

# 4) Numeric features


# 5) translate helper (same as before)
def translate_numeric_conj(conj: Conjecture) -> Conjecture:
    ineq = conj.conclusion
    H, op = conj.hypothesis, ineq.op
    if op in (">=", ">"):   return Conjecture(H, T_bool)
    if op in ("<=", "<"):   return Conjecture(H, ~T_bool)
    return None


features = [
    pg.min_degree,
    pg.max_degree,
    pg.matching_number,
    pg.order
]
# 6) Generate numeric conjectures on both T_int and its inverse
candidates = []
for target_prop in (T_int, T_inv_int):
    for gen in (ratios, convex_hull, linear_programming):
        for conj in gen(
            df,
            features   = features,
            target     = target_prop,
            hypothesis = pg.base
        ):
            if not conj.is_true(df):
                continue
            bc = translate_numeric_conj(conj)
            if bc is not None:
                candidates.append(bc)

# 7) Sophie‐filter
accepted = []
for c in candidates:
    if sophie_accept(c, accepted, df, min_fraction=0.1):
        accepted.append(c)

# 8) Print results
for i, conj in enumerate(accepted, start=1):
    print(f"Conjecture {i}: {pg.forall(conj)}   acc={conj.accuracy(df):.2%}")


In [ ]:
candidates

In [ ]:
import pandas as pd
import itertools
from txgraffiti.example_data            import graph_data
from txgraffiti.logic.conjecture_logic  import Predicate, Conjecture, TRUE

# graph_data.drop(columns=['connected'], inplace=True)

def boolean_hypotheses(
    df: pd.DataFrame,
    *,
    base: Predicate,
    target: Predicate,
    max_depth: int = 2
):
    """
    Enumerate all Boolean‐predicate hypotheses H up to `max_depth` Boolean
    connectives, conjoined with `base`, but *exclude* trivial or degenerate
    H that are identical to `target`, its negation, `base`, or `TRUE`.
    Yields each H as a Predicate.
    """
    # 1) collect atomic Predicates: base plus every bool column
    atoms = [base]
    for col in df.columns:
        if df[col].dtype == bool:
            atoms.append(Predicate(col, lambda df, c=col: df[c]))

    # 2) recursive expansion of ¬ and ∧ up to max_depth
    def expand(current, depth):
        if depth == 0:
            yield from current
        else:
            # add negations
            negs = [~p for p in current]
            # add pairwise conjunctions
            conjs = [p & q for p, q in itertools.combinations(current + negs, 2)]
            yield from expand(current + negs + conjs, depth - 1)

    seen = set()
    for p in expand(atoms, max_depth):
        # 3) normalize name for deduplication
        key = p.name
        if key in seen:
            continue
        seen.add(key)

        # 4) skip trivial or degenerate hypotheses
        if p == target or p == ~target or p == base or p == TRUE:
            continue
        mask = p(df)
        # skip if hypothesis never holds
        if not mask.any():
            continue

        yield p


# -----------------------------------------------------------------------------
# 1) Define the Dalmatian/Sophie heuristic (no playground)
# -----------------------------------------------------------------------------
def larson(
    new_conj: Conjecture,
    existing: list[Conjecture],
    df:       pd.DataFrame,
    *,
    kind: str = "sufficient"
) -> bool:
    # must hold on all rows
    if not new_conj.is_true(df):
        return False

    # compare only those with same conclusion predicate
    same = [c for c in existing if c.conclusion == new_conj.conclusion]
    if not same:
        return True

    new_mask = new_conj.hypothesis(df)
    old_masks = [c.hypothesis(df) for c in same]

    if kind == "sufficient":
        union_old = pd.concat(old_masks, axis=1).any(axis=1)
        diff = new_mask & ~union_old
        return bool(diff.any())
    else:
        new_neg = ~new_mask
        old_neg = [~m for m in old_masks]
        union_old_neg = pd.concat(old_neg, axis=1).any(axis=1)
        diff = new_neg & ~union_old_neg
        return bool(diff.any())

# register under the name "dalmatian"
# register_heuristic("dalmatian", dalmatian_accept)

# -----------------------------------------------------------------------------
# 2) Boolean‐hypothesis generator up to depth 2 (yields Predicate H)
# -----------------------------------------------------------------------------
def boolean_hypotheses(
    df: pd.DataFrame,
    *,
    base: Predicate,
    max_depth: int = 2
):
    # collect atomic Predicates: TRUE and each boolean column
    atoms = [base]
    for col in df.columns:
        if df[col].dtype == bool:
            atoms.append(Predicate(col, lambda df, c=col: df[c]))

    def expand(current, depth):
        if depth == 0:
            yield from current
        else:
            # negations
            negs = [~p for p in current]
            # pairwise conjunctions
            conjs = [p & q for p, q in itertools.combinations(current + negs, 2)]
            yield from expand(current + negs + conjs, depth - 1)

    # start from atoms (each conjoined with base)
    for p in expand(atoms, max_depth):
        yield p

# -----------------------------------------------------------------------------
# 3) Load data and set up base hypothesis & target
# -----------------------------------------------------------------------------
df = graph_data.copy()

# base hypothesis: simple connected graphs
# base = Predicate("connected", lambda df: df["connected"])

# Boolean target: has_hamiltonian_path
target = Predicate("claw_free", lambda df: df["claw_free"])

# -----------------------------------------------------------------------------
# 4) Generate, wrap into Conjecture, and filter manually
# -----------------------------------------------------------------------------
accepted: list[Conjecture] = []

for H in boolean_hypotheses(df, base=TRUE, max_depth=2):
    # 0) filter trivial H
    if H == target or H == ~target or H == TRUE or H == base:
        continue

    conj = Conjecture(H, target)

    # 1) make sure hypothesis actually covers some rows
    if not H(df).any():
        continue

    # 2) apply Sophie
    if larson(conj, accepted, df, kind="sufficient"):
        accepted.append(conj)

# -----------------------------------------------------------------------------
# 5) Display the results
# -----------------------------------------------------------------------------
for i, c in enumerate(accepted, start=1):
    # represent as "∀ G: H → T"
    print(f"Conjecture {i}: ∀ G: ({c.hypothesis.name}) → ({c.conclusion.name})",
          f"(accuracy {c.accuracy(df):.2%})")


In [ ]:
graph_data['connected']

In [ ]:
import pandas as pd
import itertools
from txgraffiti.example_data            import graph_data
from txgraffiti.logic.conjecture_logic  import (
    Property, Predicate, Inequality, Conjecture, Constant
)

# -----------------------------------------------------------------------------
# 1) Candidate generator on a single `df`
# -----------------------------------------------------------------------------
def sophie_candidates(
    df: pd.DataFrame,
    *,
    invariants: list[Property],
    target:     Predicate,
    operators:  list[str] = ["<=", ">=", "=="],
    tol:        float   = 1e-8
):
    """
    For each ordered pair (P,Q) of `invariants` and each op in `operators`,
    form the Inequality H := (P op Q).  Compute its cover set in `df`:

        cover = { i : H(df).iloc[i] == True }.

    If for all i in cover, target(df).iloc[i] is also True, emit
        Conjecture(hypothesis=H, conclusion=target).
    """
    targ_mask = target(df)
    for P, Q in itertools.permutations(invariants, 2):
        for op in operators:
            H = Inequality(P, op, Q)              # H is a Predicate
            cover = H(df)
            if not cover.any():
                continue                          # skip empty covers
            # require cover ⊆ {rows where target is True}
            if (cover & ~targ_mask).any():
                continue
            yield Conjecture(H, target)


# -----------------------------------------------------------------------------
# 2) Sophie‐filter: novelty on the same `df`
# -----------------------------------------------------------------------------
def sophie_filter(
    new_conj: Conjecture,
    accepted: list[Conjecture],
    df:       pd.DataFrame
) -> bool:
    """
    Accept `new_conj` only if its cover set adds at least one new row
    beyond the union of all previously accepted conjectures' cover sets.
    """
    # cover set of the new conjecture
    new_cover = new_conj.hypothesis(df)

    # union of old covers
    if accepted:
        old_union = pd.concat(
            [c.hypothesis(df) for c in accepted], axis=1
        ).any(axis=1)
    else:
        old_union = pd.Series(False, index=df.index)

    # must add at least one row not already covered
    delta = new_cover & ~old_union
    return bool(delta.any())


# -----------------------------------------------------------------------------
# 3) Usage on the built-in `graph_data`
# -----------------------------------------------------------------------------
df = graph_data.copy()

# pick your Boolean target P
is_reg = Predicate("regular", lambda df: df["min_degree"] == df["max_degree"])
is_triangle_free = Predicate("triangle_free", lambda df: df["triangle_free"])

# choose numeric invariants to try
invars = [
    Property("order",          lambda df: df["order"]),
    Property("min_degree",     lambda df: df["min_degree"]),
    Property("matching_number",lambda df: df["matching_number"]),
    Property("max_degree", lambda df: df["max_degree"]),
    Property("independence_number", lambda df: df["independence_number"]),
    Constant(1),
    Constant(2),
    Constant(3),
]

# 4) generate + filter
accepted: list[Conjecture] = []
for cand in sophie_candidates(df, invariants=invars, target=is_reg | is_triangle_free):
    if sophie_filter(cand, accepted, df):
        accepted.append(cand)

# 5) display
for i, conj in enumerate(accepted, start=1):
    H, T = conj.hypothesis.name, conj.conclusion.name
    touch = conj.hypothesis(df).sum()
    total = len(df)
    print(f"{i:2d}. If {H} then {T}  "
          f"(covers {touch}/{total} graphs)")


In [ ]:
list(sophie_candidates(df, invariants=invars, target=is_triangle_free))

In [ ]:
import pandas as pd
import numpy as np
import itertools
from typing import Iterator, List

from txgraffiti.logic.conjecture_logic import (
    Property, Predicate, Constant, Conjecture
)

def arithmetic_expressions(
    invariants: List[Property],
    *,
    max_terms: int = 2,
    coeffs:     List[int] = (-2, -1, 1, 2)
) -> Iterator[Property]:
    """
    Yield all linear combinations of up to `max_terms` distinct invariants:
        E(x) = a1*P1(x) + a2*P2(x) + …
    where each ai ∈ coeffs, Pi from `invariants`.
    """
    for r in range(1, max_terms+1):
        for combo in itertools.combinations(invariants, r):
            # every choice of coefficients of length r
            for coef_tuple in itertools.product(coeffs, repeat=r):
                # skip all‐zero
                if all(a == 0 for a in coef_tuple):
                    continue
                # build expression
                name = " + ".join(
                    f"{a}*{P.name}" if a!=1 else P.name
                    for a,P in zip(coef_tuple, combo) if a!=0
                )
                def make_func(combo=combo, coefs=coef_tuple):
                    def f(df):
                        s = 0
                        for a,P in zip(coefs, combo):
                            s += a * P(df)
                        return s
                    return f

                yield Property(f"({name})", make_func())


def arithmetic_threshold_hypotheses(
    df: pd.DataFrame,
    *,
    invariants: List[Property],
    target:     Predicate,
    hypothesis: Predicate,
    max_expr_terms:   int = 2,
    coeffs:           List[int] = (-2, -1, 1, 2),
    num_thresholds:   int = 5
) -> Iterator[Conjecture]:
    """
    For each generated arithmetic expression E, compute `num_thresholds`
    quantile cut‐points over the hypothesis region, and yield
      Conjecture( hypothesis & (E ≤ c),  target )
    whenever that implication holds perfectly on df.
    """
    # restrict to hypothesis region
    mask0 = hypothesis(df)
    subdf  = df[mask0]

    # enumerate expressions
    for E in arithmetic_expressions(invariants,
                                    max_terms=max_expr_terms,
                                    coeffs=coeffs):
        # evaluate E on sub‐DataFrame
        vals = E(subdf)
        # pick a few distinct quantiles
        qs = pd.Series(vals).quantile(
            q = np.linspace(0,1,num_thresholds+2)[1:-1]
        ).unique()
        for c in qs:
            # build the cut‐predicate Hc := H & (E ≤ c)
            cut = E <= Constant(float(c))
            Hc  = hypothesis & cut
            if not Hc(df).any():
                continue
            # check perfect implication
            mask_c = Hc(df)
            if target(df)[mask_c].all():
                yield Conjecture(Hc, target)

            # optionally also try the ≥ direction
            cut2 = E >= Constant(float(c))
            Hc2  = hypothesis & cut2
            if Hc2(df).any() and target(df)[Hc2(df)].all():
                yield Conjecture(Hc2, target)


In [ ]:
from txgraffiti.example_data            import graph_data
from txgraffiti.logic.conjecture_logic  import Predicate

df = graph_data.copy()

# base hypothesis: connected graphs
base = Predicate("connected", lambda df: df["connected"])

# pick your Boolean target P
is_reg = Predicate("regular", lambda df: df["min_degree"] == df["max_degree"])
is_triangle_free = Predicate("triangle_free", lambda df: df["triangle_free"])

# choose numeric invariants to try
invars = [
    Property("order",          lambda df: df["order"]),
    Property("min_degree",     lambda df: df["min_degree"]),
    Property("matching_number",lambda df: df["matching_number"]),
    Property("max_degree", lambda df: df["max_degree"]),
    Property("independence_number", lambda df: df["independence_number"]),
]
# generate all "a·P1 + b·P2" cut‐point conjectures
candidates = list(arithmetic_threshold_hypotheses(
    df,
    invariants       = invars,
    target           = is_triangle_free,
    hypothesis       = base,
    max_expr_terms   = 2,        # combine at most 2 invariants
    coeffs           = (-2,-1,1,2),
    num_thresholds   = 4         # 4 quantile cuts each
))

# inspect a few
for conj in candidates[:5]:
    expr = conj.hypothesis.name
    print(f"If {expr} then {conj.conclusion.name},  holds? {conj.is_true(df)}")


In [ ]:
from txgraffiti.example_data            import graph_data
from txgraffiti.logic.conjecture_logic  import Predicate, TRUE
from fractions import Fraction

df = graph_data.copy()

# base hypothesis: connected graphs
base = Predicate("connected", TRUE)

# Boolean target: Hamiltonian‐path existence
# target = Predicate("is_bipartite",
#                    lambda df: df["bipartite"])


tar = "triangle_free"
target = Predicate(tar, lambda df: df[tar])

# numeric invariants to combine
invars = [
    Property("order",          lambda df: df["order"]),
    Property("min_degree",     lambda df: df["min_degree"]),
    Property("matching_number",lambda df: df["matching_number"]),
    Property("max_degree", lambda df: df["max_degree"]),
    Property("independence_number", lambda df: df["independence_number"]),
]

# generate all "a·P1 + b·P2" cut‐point conjectures
candidates = list(arithmetic_threshold_hypotheses(
    df,
    invariants       = invars,
    target           = target,
    hypothesis       = base,
    max_expr_terms   = 3,        # combine at most 2 invariants
    coeffs           = (-2,-1,1,2, 3, Fraction(3/4).limit_denominator()),
    num_thresholds   = 4         # 4 quantile cuts each
))

# 4) generate + filter
accepted: list[Conjecture] = []
for cand in candidates:
    if sophie_filter(cand, accepted, df):
        accepted.append(cand)


for i, conj in enumerate(accepted, start=1):
    H, T = conj.hypothesis.name, conj.conclusion.name
    touch = conj.hypothesis(df).sum()
    total = len(df)
    print(f"{i:2d}. If {H} then {T}  "
          f"(covers {touch}/{total} graphs)")

In [ ]:
df.triangle_free

In [ ]:
df = graph_data

df.columns.to_list()

In [ ]:
import numpy as np
from typing import Iterator, Sequence, Set
from txgraffiti.logic.conjecture_logic import Property

# ── Binary ops, as before ──
BINOPS = {
    '+': Property.__add__,
    '-': Property.__sub__,
    '*': Property.__mul__,
    '/': Property.__truediv__,
    '**': Property.__pow__,
    '%': Property.__mod__,
}

# ── Unary ops registry ──
UNARYOPS = {
    'neg':      lambda p: -p,
    'abs':      lambda p: Property(f"|{p.name}|",    lambda df, p=p: p(df).abs()),
    'sqrt':     lambda p: Property(f"sqrt({p.name})", lambda df, p=p: p(df).pow(0.5)),
    'log':      lambda p: Property(f"log({p.name})",  lambda df, p=p: p(df).apply(np.log)),
    'exp':      lambda p: Property(f"exp({p.name})",  lambda df, p=p: p(df).apply(np.exp)),
    # add floor, ceil, etc.
}

def generate_expressions(
    bases: Sequence[Property],
    max_depth: int = 3,
    allow_unary: bool = True,
) -> Iterator[Property]:
    """
    Generate all distinct Property expressions up to max_depth,
    using both unary and binary ops.
    """
    seen: Set[str] = set()

    def _recurse(current: Sequence[Property], depth: int):
        if depth >= max_depth:
            return

        next_level: Set[Property] = set()

        # ── first apply unary ops ──
        if allow_unary:
            for p in current:
                for opname, opfunc in UNARYOPS.items():
                    expr = opfunc(p)
                    if expr.name not in seen:
                        seen.add(expr.name)
                        yield expr
                        next_level.add(expr)

        # ── then apply binary ops against all bases ──
        for a in current:
            for b in bases:
                for sym, op in BINOPS.items():
                    try:
                        expr = op(a, b)
                    except Exception:
                        continue
                    if expr.name not in seen:
                        seen.add(expr.name)
                        yield expr
                        next_level.add(expr)

        # ── recurse on the newly-created expressions ──
        for expr in next_level:
            yield from _recurse([expr], depth + 1)

    # seed with the bases themselves
    for p in bases:
        seen.add(p.name)
        yield p

    # kick off
    yield from _recurse(list(bases), depth=0)


In [ ]:
from txgraffiti.logic.conjecture_logic import Property, Constant


# primitives:
deg   = Property("deg", lambda df: df["degree"])
resi  = Property("res", lambda df: df["residue"])
const = Constant(1)

for expr in generate_expressions([deg, resi], max_depth=3):
    print(expr)


In [ ]:
import itertools
import pandas as pd
from txgraffiti.logic.conjecture_logic import Inequality, Conjecture, Predicate

def sophie_candidates(
    df: pd.DataFrame,
    *,
    invariants: list[Property],
    target:     Predicate,
    operators:  list[str] = ["<=", ">=", "=="],
    max_depth:  int      = 2,
    tol:        float    = 1e-8
):
    """
    Like before, but now for *all* arithmetic expressions built
    from your primitives up to `max_depth` operations.
    """
    # 1) build every expression up to depth=max_depth
    exprs = list(generate_expressions(invariants, max_depth=max_depth))

    targ_mask = target(df)

    # 2) just loop over every ordered pair of generated expressions
    for P, Q in itertools.permutations(exprs, 2):
        for op in operators:
            H = Inequality(P, op, Q)
            cover = H(df)
            if not cover.any():
                continue
            # require cover ⊆ {rows where target is True}
            if (cover & ~targ_mask).any():
                continue
            yield Conjecture(H, target)


In [ ]:
def sophie_filter(
    new_conj: Conjecture,
    accepted: list[Conjecture],
    df:       pd.DataFrame
) -> bool:
    """
    Accept `new_conj` only if its cover set adds at least one new row
    beyond the union of all previously accepted conjectures' cover sets.
    """
    # cover set of the new conjecture
    new_cover = new_conj.hypothesis(df)

    # union of old covers
    if accepted:
        old_union = pd.concat(
            [c.hypothesis(df) for c in accepted], axis=1
        ).any(axis=1)
    else:
        old_union = pd.Series(False, index=df.index)

    # must add at least one row not already covered
    delta = new_cover & ~old_union
    return bool(delta.any())

In [ ]:
from txgraffiti.example_data import graph_data
df = graph_data.copy()

# pick your Boolean target P
is_reg = Predicate("regular", lambda df: df["min_degree"] == df["max_degree"])
is_triangle_free = Predicate("triangle_free", lambda df: df["triangle_free"])

# choose numeric invariants to try
invars = [
    Property("order",          lambda df: df["order"]),
    Property("min_degree",     lambda df: df["min_degree"]),
    Property("matching_number",lambda df: df["matching_number"]),
    Property("max_degree", lambda df: df["max_degree"]),
    Property("independence_number", lambda df: df["independence_number"]),
    Constant(1),
    Constant(2),
    Constant(3),
]

# 4) generate + filter
accepted: list[Conjecture] = []
for cand in sophie_candidates(df, invariants=invars, target=is_reg | is_triangle_free):
    if sophie_filter(cand, accepted, df):
        accepted.append(cand)

# 5) display
for i, conj in enumerate(accepted, start=1):
    H, T = conj.hypothesis.name, conj.conclusion.name
    touch = conj.hypothesis(df).sum()
    total = len(df)
    print(f"{i:2d}. If {H} then {T}  "
          f"(covers {touch}/{total} graphs)")

In [ ]:
import txgraffiti as tx

c1 = tx.Constant(1)

c2 = tx.Constant(2)

c1 + c2

In [ ]:
c1*c2

In [ ]:
c1 in [c1 + 0]

In [ ]:
c3.name

In [6]:
Constant(1) + Constant(2)

<Property (1 + 2)>

In [ ]:
import pandas as pd

df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

c = Constant(2) + Constant(5)

print(f"{c = } \n")
print(f"{c(df)}")

c = <Property (2 + 5)> 

0    7
1    7
2    7
dtype: int64


In [13]:
expr = c * 3 - 2
print(expr)

<Property (((2 + 5) * 3) - 2)>


In [ ]:

import pandas as pd
from txgraffiti.logic.conjecture_logic import Property

df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

alpha = Property('alpha', lambda df: df['alpha'])

print(f"{alpha = } \n")


print(f"object: {alpha > 1}, name: {(alpha > 1).name} \n")

print((alpha > 1)(df))

alpha = <Property alpha> 

object: <Predicate alpha > 1>, name: alpha > 1 

0    False
1     True
2     True
dtype: bool


In [20]:
both   = (alpha > 1) & (alpha < 3)    # AND
both

<Predicate (alpha > 1) ∧ (alpha < 3)>

In [27]:
df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

P = Property('alpha', lambda df: df['alpha'])
Q = Property('beta', lambda df: df['beta'])

# builds an Inequality for y ≤ 2·x + 3
ineq1 = Q <= (2 * P + 3)
print(f"{ineq1 = } \n")
print(ineq1(df), "\n")
print(ineq1.slack(df), "\n")



ineq1 = <Predicate beta <= ((2 * alpha) + 3)> 

0    True
1    True
2    True
dtype: bool 

0    2
1    6
2    8
dtype: int64 



In [30]:
from txgraffiti.logic import Conjecture
df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

P = Property('alpha', lambda df: df['alpha'])
Q = Property('beta', lambda df: df['beta'])

# builds an Inequality for y ≤ 2·x + 3
hypothesis = Predicate('connected', lambda df: df['connected'])
ineq1 = Q <= (2 * P + 3)

conj = Conjecture(hypothesis, ineq1)
print(f"{conj = } \n")
print(conj(df), "\n")
print(f"{conj.is_true(df) = }")

conj = <Conj (connected) → (beta <= ((2 * alpha) + 3))> 

0    True
1    True
2    True
dtype: bool 

conj.is_true(df) = True


In [31]:
from txgraffiti.logic import Conjecture
df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

P = Property('alpha', lambda df: df['alpha'])
Q = Property('beta', lambda df: df['beta'])

# builds an Inequality for y ≤ 2·x + 3
hypothesis = Predicate('connected', lambda df: df['connected'])
ineq1 = Q <= (2 * P + 3)

conj = hypothesis.implies(ineq1, as_conjecture=True)
print(f"{conj = } \n")
print(conj(df), "\n")
print(f"{conj.is_true(df) = }")

conj = <Conj (connected) → (beta <= ((2 * alpha) + 3))> 

0    True
1    True
2    True
dtype: bool 

conj.is_true(df) = True


In [32]:
df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

P = Property('alpha', lambda df: df['alpha'])
Q = Property('beta', lambda df: df['beta'])

# builds an Inequality for y ≤ 2·x + 3
hypothesis = Predicate('connected', lambda df: df['connected'])
ineq1 = Q <= (2 * P + 3)

conj = hypothesis >> ineq1
print(f"{conj = } \n")
print(conj(df), "\n")
print(f"{conj.is_true(df) = }")

conj = <Conj (connected) → (beta <= ((2 * alpha) + 3))> 

0    True
1    True
2    True
dtype: bool 

conj.is_true(df) = True


In [ ]:
from txgraffiti.generators import ratios

df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'gamma': [2, 3, 2],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

target = Property('alpha', lambda df: df['alpha'])
features = [
    Property('beta', lambda df: df['beta']),
    Property('gamma', lambda df: df['gamma']),
]
hypothesis = Predicate('connected', lambda df: df['connected'])
list(ratios(df, features=features, target=target, hypothesis=hypothesis))


[<Conj (connected) → (alpha >= (1/3 * beta))>,
 <Conj (connected) → (alpha <= (3 * beta))>,
 <Conj (connected) → (alpha >= (1/2 * gamma))>,
 <Conj (connected) → (alpha <= (3/2 * gamma))>]

In [39]:
from txgraffiti.generators import convex_hull
df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'gamma': [2, 3, 2],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

target = Property('alpha', lambda df: df['alpha'])
features = [
    Property('beta', lambda df: df['beta']),
    # Property('gamma', lambda df: df['gamma']),
]
hypothesis = Predicate('connected', lambda df: df['connected'])
list(convex_hull(df, features=features, target=target, hypothesis=hypothesis))

[<Conj (connected) → (alpha >= ((-1/2 * beta) + 5/2))>,
 <Conj (connected) → (alpha <= ((-1 * beta) + 4))>]

In [40]:
from txgraffiti.generators import linear_programming
df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [3, 1, 1],
        'gamma': [2, 3, 2],
        'connected': [True, True, True],
        'K_n': [True, False, False],
        'tree': [False, False, True],
    })

target = Property('alpha', lambda df: df['alpha'])
features = [
    Property('beta', lambda df: df['beta']),
    Property('gamma', lambda df: df['gamma']),
]
hypothesis = Predicate('connected', lambda df: df['connected'])
list(linear_programming(df, features=features, target=target, hypothesis=hypothesis))

[<Conj (connected) → (alpha <= (((-1 * beta) + 6) + (-1 * gamma)))>,
 <Conj (connected) → (alpha >= (((-1 * beta) + 6) + (-1 * gamma)))>]

In [ ]:
from txgraffiti.heuristics import morgan
df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [1, 1, 2],
        'connected': [True, True, True],
        'tree': [False, True, True],
    })


alpha = Property('alpha', lambda df: df['alpha'])
beta = Property('beta', lambda df: df['beta'])
ineq = alpha <= beta + 1

hyp1 = Predicate('connected', lambda df: df['connected'])
hyp2 = Predicate('tree', lambda df: df['tree'])

conj1 = hyp1 >> ineq
conj2 = hyp2 >> ineq

print(f"{morgan(conj1, [conj2], df) = } \n ")
print(f"{morgan(conj2, [conj1], df) = } \n ")

morgan(conj1, [conj2], df) = True 
 
morgan(conj2, [conj1], df) = False 
 


In [ ]:
from txgraffiti.heuristics import dalmatian
df = pd.DataFrame({
        'alpha': [1, 2, 3],
        'beta': [1, 1, 2],
        'connected': [True, True, True],
        'tree': [False, True, True],
    })


alpha = Property('alpha', lambda df: df['alpha'])
beta = Property('beta', lambda df: df['beta'])


hyp = Predicate('connected', lambda df: df['connected'])

ineq1 = alpha <= beta + 1
ineq2 = alpha <= beta + 2

conj1 = hyp >> ineq1
conj2 = hyp >> ineq2

print(f"{dalmatian(conj1, [conj2], df) = } \n ")
print(f"{dalmatian(conj2, [conj1], df) = } \n ")

dalmatian(conj1, [conj2], df) = True 
 
dalmatian(conj2, [conj1], df) = False 
 
